# Python - get csv for library updates

run in same directory as conda list > live_tag.txt files

to get the text files with conda list output run:

<code>conda list --export > filename.txt</code>

for example, on live release RStudio notebook run:

<code>conda list --export > live_rstudio.txt</code>

Ensure live and beta versions exist for all notebooks.

Ensure all txt files have been uploaded to the directory containing this notebook.

In [1]:
import csv

In [2]:
# types of notebook  -- add to this if more languages incorporated
tags = ["std", "geo", "haskell", "julia", "py2", "stan", "rstudio", "sage"]

# filenames of format: version_tag.txt where version is either live or beta

In [3]:
def getLists(tag):

    # put live file in list of lines
    liveFile =  "live_" + str(tag) + ".txt"
    lFile = open(liveFile, "r")
    lList = lFile.readlines()[3:]

    # put beta file in list of lines
    betaFile = "beta_" + str(tag) + ".txt"
    bFile = open(betaFile, "r")
    bList = bFile.readlines()[3:]
    
    return lList, bList

In [4]:
# take list of lines
# return dictionary of (library, version) pairs
def divFile(lineList):
    vs = dict()
    for line in lineList:
        name, version, _ = line.split("=")
        vs[name] = version
    return vs

In [5]:
# rows for all libraries
def buildRows(lDict, bDict):

    csvRows = set()
    for lib in bothVs:
        if (lDict[lib] == bDict[lib]):
            csvRows.add((lib, lDict[lib], "same"))
        else:
            csvRows.add((lib, lDict[lib], bDict[lib]))
        
    for lib in liveOnly:
        csvRows.add((lib, lDict[lib], "DNE"))
    
    for lib in betaOnly:
        csvRows.add((lib, "DNE", bDict[lib]))

    csvRowsL = list(csvRows)
    csvRowsL = sorted(csvRowsL, key=lambda tup: tup[0])
    
    return csvRowsL

In [6]:
# rows for only libraries that differ between live and beta
def buildRows2(lDict, bDict):

    csvRows = set()
    for lib in bothVs:
        if (lDict[lib] != bDict[lib]):
            csvRows.add((lib, lDict[lib], bDict[lib]))
        
    for lib in liveOnly:
        # encode missing as DNE - does not exist
        csvRows.add((lib, lDict[lib], "DNE"))
    
    for lib in betaOnly:
        # encode missing as DNE - does not exist
        csvRows.add((lib, "DNE", bDict[lib]))

    csvRowsL = list(csvRows)
    csvRowsL = sorted(csvRowsL, key=lambda tup: tup[0])
    
    return csvRowsL

In [7]:
# takes text file output of conda list
# write csv files - nameOfNotebook.csv containing:
# name of library
# version on live
# version on beta (if different from live version)

for tag in tags:
    
    lList, bList = getLists(tag)
    
    lDict = divFile(lList)
    bDict = divFile(bList)
    
    liveOnly = lDict.keys() - bDict.keys()
    betaOnly = bDict.keys() - lDict.keys()
    bothVs   = list(lDict.keys() & bDict.keys())
    
    rows = buildRows2(lDict, bDict)
    
    fcsv = str(tag) + ".csv"
    with open(fcsv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["library", "live", "toBeta"])
    
        for row in rows:
            writer.writerow(list(row))